In [ ]:
%%writefile train.py

import argparse
import os

from sklearn.externals import joblib

from resnet152 import ResNet152

if __name__ == '__main__':
    # Define the arguments.
    parser = argparse.ArgumentParser(description='Fit and evaluate a model based on train-test datasets.')
    parser.add_argument('--outputs', help='the outputs directory', default='.')
    parser.add_argument('--model', help='the model file', default='model.pkl')
    args = parser.parse_args()

    resnet_152 = ResNet152(weights="imagenet")

    # Save the model to a file, and report on its size.
    os.makedirs(args.outputs, exist_ok=True)
    joblib.dump(resnet_152.get_weights(), os.path.join(args.outputs, args.model))



In [ ]:
%%writefile score.py

from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from azureml.contrib.services.aml_request import rawhttp
from azureml.core.model import Model
from azureml.contrib.services.aml_response import AMLResponse
from toolz import compose
import numpy as np
from PIL import Image, ImageOps

_NUMBER_RESULTS = 3

def init():
    """ Initialise the model and scoring function
    """
    global model
    model = ResNet152()
    model_name = "resnet_model"
    model_path = Model.get_model_path(model_name)    
    model.load_weights(model_path)

@rawhttp
def run(request):
    """ Make a prediction based on the data passed in using the preloaded model
    """
    if request.method == 'POST':
        """ Classify the input using the loaded model
        """
        #preprocess
        transform_input = compose(_pil_to_numpy, _image_ref_to_pil_image)
        transformed_dict = {key: transform_input(img_ref) for key, img_ref in request.files.items()}
        img_array = preprocess_input(np.stack(list(transformed_dict.values())))
        
        preds = model.predict(img_array)        
        
        # Postprocess
        # nverting predictions to float64 since we are able to serialize float64 but not float32
        preds = decode_predictions(preds.astype(np.float64), top=_NUMBER_RESULTS)
        return dict(zip(transformed_dict.keys(), preds))

    if request.method == 'GET':
        return {"azEnvironment": "Azure"}
    return AMLResponse("bad request", 500)

                    
def _image_ref_to_pil_image(image_ref):
    """ Load image with PIL (RGB)
    """
    return Image.open(image_ref).convert("RGB")


def _pil_to_numpy(pil_image):
    img = ImageOps.fit(pil_image, (224, 224), Image.ANTIALIAS)
    img = image.img_to_array(img)
    return img


In [ ]:
from azure_utils.machine_learning.ai_workspace import  DeepRealtimeScore

DeepRealtimeScore.get_or_or_create(project_config="dl-rts.project.yml")

Notebook Complete